In [454]:
import math
import pandas as pd
import numpy as np
from pylab import *
import random
mpl.rcParams['font.sans-serif'] = ['SimHei']  # 添加这条可以让图形显示中文

In [455]:
# 2变换法
def TSP_2_reverse(df,i,j):
    df = df.copy()
    for idx in range(i,int((j-i+1)/2)+i):
        df_tmp = df.loc[idx]
        df.loc[idx] = df.loc[j-(idx-i)]
        df.loc[j-(idx-i)] = df_tmp
    return df

In [456]:
# 获取点对点的距离
def get_2p_dis(df,i,j):
    return math.sqrt( (df["lo"].loc[i]-df["lo"].loc[j])**2 + (df["la"].loc[i]-df["la"].loc[j])**2 )

In [457]:
# 获取总的路径花费
def get_total_cost(df,df_city_map):
    res = 0
    for i in range(len(df) -1):
        res += df_city_map[df["city"].loc[i]][df["city"].loc[i+1]]
    res += df_city_map[df["city"].loc[len(df)-1]][df["city"].loc[0]]
    return res

In [458]:
# 获取总的路径花费
def get_total_list_cost(city_list,df_city_map):
    res = 0
    for i in range(len(df) -1):
        res += df_city_map[city_list[i]][city_list[i+1]]
    res += df_city_map[city_list[len(df)-1]][city_list[0]]
    return res

In [459]:
# 预处理城市之间两两距离
def pre_deal(df):
    df_len = len(df)
    city_map_dict = {}
    for i in range(df_len):
        city_mul = {}
        for j in range(df_len):
            city_mul[df.loc[j]["city"]] = get_2p_dis(df,i,j)
        city_map_dict[df.loc[i]["city"]] = city_mul
    return pd.DataFrame.from_dict(city_map_dict)

In [460]:
# 数据读入
def read_data():
    df = pd.DataFrame(columns = ["city", "lo", "la"]) #创建一个空的dataframe
    with open("TSP.txt","r",encoding="utf-8") as f:
        for line in f.readlines():
            city,lo,la = line.replace("\n","").split(" ")
    #         print(city,lo,la)
            df = df.append({"city":city,"lo":float(lo),"la":float(la)},ignore_index=True)
    return df

In [461]:
# 画图
def plot_city_route(df):
    plt.clf()
    # plot中参数的含义分别是横轴值，纵轴值，线的形状，颜色，透明度,线的宽度和标签
    plt.plot(df["lo"], df["la"], 'ro-', color='#FF0000', alpha=0.8, linewidth=1, label='city')
    plt.legend(loc="lower left")
    plt.xlabel('经度')
    plt.ylabel('纬度')

In [462]:
def lower_bound(nums, target):
    low, high = 0, len(nums)-1
    pos = len(nums)
    while low<high:
        mid = int((low+high)/2)
        if nums[mid] < target:
            low = mid+1
        else:#>=
            high = mid
            #pos = high
    if nums[low]>=target:
        pos = low
    return pos

In [463]:
# 双链杂交
def hybrid_route(ra, rb, p_cro):
    
    # 随机确定杂交起始位置
    cro_pos = random.randint(0,len(ra)-1)
    
    for cur_pos in range(cro_pos,len(ra)):
        # 杂交概率
        if(random.random()<=p_cro):
            city_a = ra[cur_pos]
            city_b = rb[cur_pos] 
            if(city_a == city_b):
                continue
            
            # city a 在 list b 中的位置
            b_city_a_pos = rb.index(city_a)
            # city b 在 list a 中的位置
            a_city_b_pos = ra.index(city_b)   
            
            # 交换城市
            tmp_city = rb[cur_pos]
            rb[cur_pos] = ra[cur_pos]
            ra[cur_pos] = tmp_city
            
            # 防止重复，相应城市也要交换
            tmp_city = rb[b_city_a_pos]
            rb[b_city_a_pos] = ra[a_city_b_pos]
            ra[a_city_b_pos] = tmp_city
            
        else:
            continue

In [464]:
# 变异
def mut_route(route, p_mut):
    for i in range(len(route)):
        if(random.random()<=p_mut):
            change_pos = random.randint(0, len(route)-1)
            if i == change_pos:
                continue
            else:
                # 变异，即随机与另一个城市交换位置
                tmp_city = route[change_pos]
                route[change_pos] = route[i]
                route[i] = tmp_city


In [465]:
if __name__ == "__main__":
    N = 200
    M = 200
    p_cro = 0.8
    p_mut = 0.1

    # 数据读入，数据预处理
    df = read_data()
    df_city_map = pre_deal(df)
    df_cur = df
    # 去重set
    df_city_group = set()
    while len(df_city_group)<N:
        u = random.randint(0,len(df_cur)-1-1)
        v = random.randint(u+1,len(df_cur)-1)
        # 逆序uv路径
        df_cur = TSP_2_reverse(df_cur,u,v)
        # 生成初始种群
        df_city_group.add(tuple(list(df_cur["city"])))

    # plot_city_route(df_cur)

In [466]:
    # 初代数据预处理
    df_city_group_cost_dict = pd.DataFrame(columns = ["route", "cost", "p_sel"])
    for item in df_city_group:
        cur_city_list_cost = get_total_list_cost(list(item),df_city_map)
        df_city_group_cost_dict = df_city_group_cost_dict.append({"route":item,
                                                                  "cost":cur_city_list_cost,
                                                                  "p_sel":1.0/cur_city_list_cost},
                                                                 ignore_index=True)

In [474]:
    loss_str_output = ""
    for epoch in range(200000):
    # 轮盘概率预处理
        p_sel_sum = [0 for _ in range(len(df_city_group_cost_dict))]
        for i in range(len(df_city_group_cost_dict)):
            if(i==0):
                p_sel_sum[i] = df_city_group_cost_dict.iloc[i]["p_sel"]
            else:
                p_sel_sum[i] = p_sel_sum[i-1] + df_city_group_cost_dict.iloc[i]["p_sel"]

        total_p_sum = df_city_group_cost_dict["p_sel"].sum()
        
        # 选父代杂交
        df_city_group_new = set()
        while len(df_city_group_new)<M:
            cur_p = random.uniform(0,total_p_sum)
            cur_pos_a = lower_bound(p_sel_sum, cur_p)
            cur_list_a = list(df_city_group_cost_dict.iloc[cur_pos_a]["route"])

            cur_p = random.uniform(0,total_p_sum)
            cur_pos_b = lower_bound(p_sel_sum, cur_p)
            cur_list_b = list(df_city_group_cost_dict.iloc[cur_pos_b]["route"])

            # 双route杂交
            hybrid_route(cur_list_a, cur_list_b, p_cro)
            
            # 两个新子代变异
            mut_route(cur_list_a, p_mut)
            mut_route(cur_list_b, p_mut)
            
            # 生成子代入set
            df_city_group_new.add(tuple(cur_list_a))
            df_city_group_new.add(tuple(cur_list_b))

        # 子代群落
        df_city_group_new = set(list(df_city_group_new)[:200])
        
        #处理子代数据，丢弃父代数据
        df_city_group_cost_dict=df_city_group_cost_dict.drop(index=df_city_group_cost_dict.index)
        for item in df_city_group_new:
            cur_city_list_cost = get_total_list_cost(list(item),df_city_map)
            df_city_group_cost_dict = df_city_group_cost_dict.append({"route":item,
                                                                      "cost":cur_city_list_cost,
                                                                      "p_sel":1.0/cur_city_list_cost},
                                                                     ignore_index=True)
        
        # 输出群落中的最优解
        loss_str_output += str(epoch) + " " + str(df_city_group_cost_dict["cost"].min())+"\n"
        if((epoch+1)%1000==0):
            min_cost_pos = list(df_city_group_cost_dict["cost"]).index(df_city_group_cost_dict["cost"].min())
            print(df_city_group_cost_dict.iloc[min_cost_pos])
            with open("./loss.txt","a+") as f:
                f.write(loss_str_output)
            f.close()
            loss_str_output = ""

route    (兰州, 合肥, 台湾, 西安, 广州, 南昌, 上海, 石家庄, 拉萨, 西藏, 南京, ...
cost                                               393.219
p_sel                                           0.00254311
Name: 128, dtype: object
route    (长春, 海口, 澳门, 重庆, 哈尔滨, 南京, 石家庄, 郑州, 太原, 西藏, 合肥,...
cost                                               384.289
p_sel                                           0.00260221
Name: 112, dtype: object
route    (沈阳, 太原, 兰州, 成都, 郑州, 南京, 贵阳, 天津, 石家庄, 内蒙古, 宁夏,...
cost                                               379.856
p_sel                                           0.00263258
Name: 179, dtype: object
route    (西宁, 广西, 西藏, 拉萨, 澳门, 台湾, 海口, 重庆, 南昌, 郑州, 南京, 广...
cost                                               390.916
p_sel                                           0.00255809
Name: 175, dtype: object
route    (香港, 西安, 长沙, 贵阳, 宁夏, 广州, 太原, 武汉, 济南, 石家庄, 郑州, ...
cost                                               366.886
p_sel                                           0.00272564
Name: 194, dtyp

route    (台湾, 杭州, 长春, 石家庄, 天津, 沈阳, 武汉, 合肥, 南昌, 宁夏, 新疆, ...
cost                                                400.82
p_sel                                           0.00249489
Name: 42, dtype: object
route    (上海, 长春, 沈阳, 北京, 广西, 南昌, 武汉, 郑州, 成都, 西安, 澳门, 哈...
cost                                               402.558
p_sel                                           0.00248411
Name: 156, dtype: object
route    (成都, 沈阳, 杭州, 西安, 武汉, 福州, 长春, 郑州, 广西, 澳门, 广州, 台...
cost                                                360.91
p_sel                                           0.00277077
Name: 110, dtype: object
route    (上海, 武汉, 济南, 太原, 石家庄, 西藏, 新疆, 西宁, 贵阳, 宁夏, 北京, ...
cost                                               337.876
p_sel                                           0.00295967
Name: 161, dtype: object


KeyboardInterrupt: 

In [ ]:
df_loss = pd.DataFrame(columns = ["loss"]) #创建一个空的dataframe
with open("./loss.txt","r",encoding="utf-8") as f:
    for line in f.readlines():
        epoch,loss = line.replace("\n","").split(" ")
        df_loss = df_loss.append({"epoch":epoch,"loss":float(loss)},ignore_index=True)
f.close()

# loss变化情势
plt.clf()
plt.plot(df_loss["epoch"], df_loss["loss"], 'ro-', color='#FF0000', alpha=0.8, linewidth=1, label='loss')
plt.legend(loc="upper left")
plt.xlabel('epoch')
plt.ylabel('loss')